In [1]:
disp ('executing gpml startup script...')
mydir = fileparts (mfilename ('fullpath'));                 % where am I located
addpath (mydir)
dirs = {'cov','doc','inf','lik','mean','prior','util'};           % core folders
for d = dirs, addpath (fullfile (mydir, d{1})), end
dirs = {{'util','minfunc'},{'util','minfunc','compiled'}};     % minfunc folders
for d = dirs, addpath (fullfile (mydir, d{1}{:})), end
% addpath([mydir,'/util/sparseinv'])
pkg load statistics
graphics_toolkit ('gnuplot')

warning('off','all');

rand('state',123456)
exp_name = 'ii2_kin40k_temp10';

executing gpml startup script...


%load(exp_name)

In [2]:
nn = 10000; 
nt = 30000;
nv = nn*0.1
n = nn-nv
[ori_xvec,ori_yvec,ori_xvec_test,ori_yvec_test] = load_data('data/kin40k','kin40k');
R = randperm(nn);
ori_xvec_val = ori_xvec(R(1:nv),:);
ori_yvec_val = ori_yvec(R(1:nv));
ori_xvec(R(1:nv),:) = [];
ori_yvec(R(1:nv)) = [];

nv =  1000
n =  9000


In [3]:
ttcs = 600;
ttm = round(n / (ttcs/2));
% fast_induce_step = 50;
induce_step = 100;

xnorm = 1; ynorm = 1;

In [4]:
% input normalization
if xnorm == 1
    norm_xmean = mean(ori_xvec);
    norm_xstd = std(ori_xvec);
    xvec = (ori_xvec - repmat(norm_xmean,n,1)) ./ repmat(norm_xstd,n,1);    
    xvec_test = (ori_xvec_test - repmat(norm_xmean,nt,1)) ./ repmat(norm_xstd,nt,1);
    xvec_val = (ori_xvec_val - repmat(norm_xmean,nv,1)) ./ repmat(norm_xstd,nv,1);
else
    xvec = ori_xvec;
    xvec_test = ori_xvec_test;
    xvec_val = ori_xvec_val;
end

In [5]:
% output normalization
if ynorm == 1
    norm_fmean = mean(ori_yvec);
    norm_fstd = std(ori_yvec);
    yvec = (ori_yvec - norm_fmean) / norm_fstd;
    yvec_test = (ori_yvec_test - norm_fmean) / norm_fstd;
    yvec_val = (ori_yvec_val - norm_fmean) / norm_fstd;
else
    yvec = ori_yvec;
    yvec_test = ori_yvec_test;
    yvec_test_val = ori_yvec_val;
end

In [6]:
grls = 0.6:0.01:1.0; % percentage of remaining data to be test

In [7]:
sf2 = 1 ; ell = 1 ; sn2 = 0.1 ; 
d = size(xvec,2);
opts.Xnorm = 'N' ; opts.Ynorm = 'N' ;
opts.ell = ell ; opts.sf2 = sf2 ; opts.sn2 = sn2 ;
opts.meanfunc = []; opts.covfunc = @covSEard; opts.likfunc = @likGauss; opts.inffunc = @infGaussLik ;

meanfunc = [];                    % empty: don't use a mean function
covfunc = opts.covfunc;              % Squared Exponental covariance function
likfunc = opts.likfunc;              % Gaussian likelihood
inffunc = opts.inffunc;
km_iters = 1e4;
opts.induce_step = induce_step ;

In [8]:
kti = 1;
grbcm0_smse_rec = zeros(kti, 1);
vfe0_smse_rec = zeros(kti, 1);
grbcm0_msll_rec = zeros(kti, 1);
vfe0_msll_rec = zeros(kti, 1);

grbcm_smse_rec = zeros(kti, 1);
vfe_smse_rec = zeros(kti, 1);
grbcm_msll_rec = zeros(kti, 1);
vfe_msll_rec = zeros(kti, 1);

In [9]:
% use validation data to find the optimal gr
best_msll = inf;
best_msll_gr = 1;
best_smse = inf;
best_smse_gr = 1;
ki = 1;

# find ratio

In [10]:
dcs = round(ttcs/2);
m = ttm;

n_per = dcs ; % size of Dc

In [11]:
Indics = randperm(n) ;
I_com = Indics(1:n_per) ; % randomly select communication set
[idx, C] = kmeans(xvec, m,'MaxIter',km_iters);    

% hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];    
opts.numOptFC = 30 ;
opts.Ms = m+1;
opts.xvec = xvec;
opts.yvec = yvec;
opts.induce_size = dcs;
opts.grbcm_baseline = 0;
opts.global_index = ones(n,1);
opts.I_com = I_com;
% opts.inffunc = @infGaussLik; opts.meanfunc = meanfunc; opts.likfunc = likfunc;
opts.covfunc = covfunc;
covfuncF = {@apxSparse, {opts.covfunc}, xvec(I_com,:)};
opts.covfuncF = covfuncF;
opts.compute_hyp = 0;

In [12]:
g_opts = opts;
g_opts.compute_hyp = 1;
g_opts.grbcm_baseline = 1;
g_opts.global_index = ones(n,1);
g_models = aggregation_train_GRBCM_VS_apx(xvec,yvec,idx,g_opts); 
opts.hyp = g_models{1}.hyp;
g_opts.hyp = g_models{1}.hyp;
[tmu,ts2, ~] =  aggregation_predict(xvec_val,g_models,'GRBCM', 1, g_opts);
if ynorm==1
    tmu = tmu * norm_fstd + norm_fmean;
    ts2 = ts2 * norm_fstd^2;
end
[grbcmMSE,grbcmSMSE,grbcmMSLL] = evaluate2(ori_xvec, ori_yvec, ori_xvec_val, ori_yvec_val, tmu, ts2);
fprintf('%s (Dc size %d): MSE %6.8f, SMSE %6.8f, MSLL %6.8f\r\n', 'GRBCM', n_per, grbcmMSE,grbcmSMSE,grbcmMSLL);
% grbcm0_smse_rec(ki) = grbcmSMSE; grbcm0_msll_rec(ki) = grbcmMSLL;
g_opts.compute_hyp = 0;

Optimizing hyps in training...
Linesearch     30;  Value 2.547982e+03
GRBCM (Dc size 300): MSE 0.03155786, SMSE 0.02997381, MSLL -1.84029091


In [13]:
vfe_opts = opts;
vfe_opts.induce_type = 'VFE_opt';
xu = xvec(I_com, :);
inffunc = @(varargin) infGaussLik(varargin{:}, struct('s', 0.0));
vfe_hyp = opts.hyp;
vfe_hyp.xu = xu;
[vfe_hyp, tmp_nlzs] = minimize(vfe_hyp,@sp_gp,-vfe_opts.induce_step,inffunc,meanfunc,covfuncF,likfunc,xvec,yvec);
vfe_opts.hyp = opts.hyp;
vfe_opts.xu = vfe_hyp.xu;
vfe_opts.inffunc = @infGaussLik; vfe_opts.meanfunc = meanfunc; vfe_opts.covfuncF = covfuncF; vfe_opts.likfunc = likfunc;
vfe_opts.covfunc = covfunc;
[tmu, ts2] = gp(vfe_hyp, @infGaussLik, meanfunc, covfuncF, likfunc, xvec, yvec, xvec_val);
if ynorm==1
    tmu = tmu * norm_fstd + norm_fmean;
    ts2 = ts2 * norm_fstd^2;
end
[vfeMSE,vfeSMSE,vfeMSLL] = evaluate2(ori_xvec, ori_yvec, ori_xvec_val, ori_yvec_val, tmu, ts2);
fprintf('%s (Dc size %d): MSE %6.8f, SMSE %6.8f, MSLL %6.8f\r\n', 'VFE', n_per, vfeMSE,vfeSMSE,vfeMSLL);
% vfe0_smse_rec(ki) = vfeSMSE; vfe0_msll_rec(ki) = vfeMSLL;
[yu, su] = gp(vfe_hyp, @infGaussLik, meanfunc, covfuncF, likfunc, xvec, yvec, vfe_opts.xu);
vfe_opts.yu = yu; vfe_opts.su = su; 

Function evaluation     99;  Value 2.269995e+05
VFE (Dc size 300): MSE 0.04606521, SMSE 0.04375296, MSLL -1.26864414


In [14]:
models = aggregation_train_GRBCM_VS_apx(xvec,yvec,idx,vfe_opts); % use hyp of vfe
[tmu,ts2] =  aggregation_predict_GRBCM_VS_apx(xvec_val,models,vfe_opts);
if ynorm==1
    tmu = tmu * norm_fstd + norm_fmean;
    ts2 = ts2 * norm_fstd^2;
end
[MSE,SMSE,MSLL] = evaluate2(ori_xvec, ori_yvec, ori_xvec_val, ori_yvec_val, tmu, ts2);
fprintf('%s (Dc size %d): \r\nMSE %6.8f, SMSE %6.4f, MSLL %6.4f\r\n', 'GRBCM++ (VFE)', n_per, MSE,SMSE,MSLL);

In [15]:
sig_temp = 10;
dcs_ecs_r = sigmoid(-MSLL+grbcmMSLL, sig_temp)
dcs = round(ttcs*dcs_ecs_r)
ecs = ttcs - dcs
m = round(n / ecs)

dcs_ecs_r =  0.70453
dcs =  423
ecs =  177
m =  51


# New unbalanced

In [16]:
n_per = dcs ; % size of Dc
Indics = randperm(n) ;
I_com = Indics(1:n_per) ; % randomly select communication set
[idx, C] = kmeans(xvec, m,'MaxIter',km_iters);    

% hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];    
opts.numOptFC = 30 ;
opts.Ms = m+1;
opts.xvec = xvec;
opts.yvec = yvec;
opts.induce_size = dcs;
opts.grbcm_baseline = 0;
opts.global_index = ones(n,1);
opts.I_com = I_com;
% opts.inffunc = @infGaussLik; opts.meanfunc = meanfunc; opts.likfunc = likfunc;
opts.covfunc = covfunc;
covfuncF = {@apxSparse, {opts.covfunc}, xvec(I_com,:)};
opts.covfuncF = covfuncF;
opts.compute_hyp = 0;

In [17]:
g_opts = opts;
g_opts.compute_hyp = 0;
g_opts.grbcm_baseline = 1;
g_opts.global_index = ones(n,1);
g_models = aggregation_train_GRBCM_VS_apx(xvec,yvec,idx,g_opts); 
opts.hyp = g_models{1}.hyp;
g_opts.hyp = g_models{1}.hyp;
[tmu,ts2, ~] =  aggregation_predict(xvec_test,g_models,'GRBCM', 1, g_opts);
if ynorm==1
    tmu = tmu * norm_fstd + norm_fmean;
    ts2 = ts2 * norm_fstd^2;
end
[grbcmMSE,grbcmSMSE,grbcmMSLL] = evaluate2(ori_xvec, ori_yvec, ori_xvec_test, ori_yvec_test, tmu, ts2);
fprintf('%s (Dc size %d): MSE %6.8f, SMSE %6.8f, MSLL %6.8f\r\n', 'GRBCM', n_per, grbcmMSE,grbcmSMSE,grbcmMSLL);
% grbcm0_smse_rec(ki) = grbcmSMSE; grbcm0_msll_rec(ki) = grbcmMSLL;
g_opts.compute_hyp = 0;

In [18]:
% % VFE Baseline
vfe_opts = opts;
vfe_opts.induce_type = 'VFE_opt';
xu = xvec(I_com, :);
vfe_hyp = opts.hyp;
vfe_hyp.xu = xu;
vfe_hyp = minimize(vfe_hyp,@sp_gp,-vfe_opts.induce_step,inffunc,meanfunc,covfuncF,likfunc,xvec,yvec);
vfe_opts.hyp = opts.hyp;
vfe_opts.xu = vfe_hyp.xu;
vfe_opts.inffunc = @infGaussLik; vfe_opts.meanfunc = meanfunc; vfe_opts.covfuncF = covfuncF; vfe_opts.likfunc = likfunc;
vfe_opts.covfunc = covfunc;
[tmu, ts2] = gp(vfe_hyp, @infGaussLik, meanfunc, covfuncF, likfunc, xvec, yvec, xvec_test);
if ynorm==1
    tmu = tmu * norm_fstd + norm_fmean;
    ts2 = ts2 * norm_fstd^2;
end
[vfeMSE,vfeSMSE,vfeMSLL] = evaluate2(ori_xvec, ori_yvec, ori_xvec_test, ori_yvec_test, tmu, ts2);
fprintf('%s (Dc size %d): MSE %6.8f, SMSE %6.8f, MSLL %6.8f\r\n', 'VFE baseline', n_per, vfeMSE,vfeSMSE,vfeMSLL);
% vfe0_smse_rec(ki) = vfeSMSE; vfe0_msll_rec(ki) = vfeMSLL;
[yu, su] = gp(vfe_hyp, @infGaussLik, meanfunc, covfuncF, likfunc, xvec, yvec, vfe_opts.xu);
vfe_opts.yu = yu; vfe_opts.su = su; 

Function evaluation     99;  Value 1.595376e+05
VFE baseline (Dc size 423): MSE 0.03744784, SMSE 0.03775445, MSLL -1.38902409


In [19]:
sp_opts = opts;
sp_opts.induce_type = 'SPGP_opt';
hyp_init(1:d,1) = -2*opts.hyp.cov(1:d);
hyp_init(d+1,1) = 2*opts.hyp.cov(d+1);
hyp_init(d+2,1) = 2*opts.hyp.lik;

sp_opts.induce_size = dcs;
xu = xvec(I_com, :);
w_init = [reshape(xu,sp_opts.induce_size*d,1);hyp_init];
[w,f] = minimize(w_init,'spgp_lik_nohyp',-sp_opts.induce_step,yvec,xvec,sp_opts.induce_size);
xb = reshape(w(1:sp_opts.induce_size*d,1),sp_opts.induce_size,d);
sp_opts.xu = xb;
sp_opts.sp_hyp = w(sp_opts.induce_size*d+1:end,1);
sp_opts.hyp = opts.hyp;

[tmu,ts2] = spgp_pred(sp_opts.yvec,sp_opts.xvec,sp_opts.xu,xvec_test,sp_opts.sp_hyp);
if ynorm==1
    tmu = tmu * norm_fstd + norm_fmean;
    ts2 = ts2 * norm_fstd^2;
end
[spgpMSE,spgpSMSE,spgpMSLL] = evaluate2(ori_xvec, ori_yvec, ori_xvec_test, ori_yvec_test, tmu, ts2);
fprintf('%s (Dc size %d): MSE %6.8f, SMSE %6.8f, MSLL %6.8f\r\n', 'SPSG baseline', n_per, spgpMSE,spgpSMSE,spgpMSLL);
% spgp0_smse_rec(ki) = spgpSMSE; spgp0_msll_rec(ki) = spgpMSLL;
[yu,su] = spgp_pred(sp_opts.yvec,sp_opts.xvec,sp_opts.xu,sp_opts.xu,sp_opts.sp_hyp);
sp_opts.yu = yu; sp_opts.su = su;

Function evaluation     99;  Value 6.125995e+02
SPSG baseline (Dc size 423): MSE 0.04365615, SMSE 0.04401359, MSLL -1.40805924


In [20]:
disp('validation results')

validation results


In [21]:
% compute informatic importance
ds = zeros(n, m);
for i=1:n
    for j=1:m
        %ds(i,j) = muti1(i)-muti2(j,i)+muti1(j)-muti2(i,j);
        ds(i,j) = norm(xvec(i, :) - C(j,:));
    end
end
% 1st closest cluster centers
[~, fcc] = min(ds, [], 2);
% set to inf
for i=1:n
    ds(i, fcc(i)) = 1e10;
end
% 2st closest cluster centers
[~, scc] = min(ds, [], 2);
rk = zeros(n, 1);
for iik=1:n
    if mod(iik, 1e2)==0
        fprintf('processing distance: %d/%d\n', iik, n);
    end
    i = fcc(iik);
    [tmp_mu, tmp_sig2] = gp(opts.hyp,inffunc,meanfunc, ...
        covfunc,likfunc,g_models{i+1}.X,g_models{i+1}.Y,xvec(iik,:));
    h1 = 0.5*log(tmp_sig2);
    j = scc(iik);
%     [tmp_mu, tmp_sig2] = gp(opts.hyp,inffunc,meanfunc, ...
%         covfunc,likfunc,[g_models{i+1}.X;g_models{j+1}.X],[g_models{i+1}.Y;g_models{j+1}.Y],xvec(iik,:));
    [tmp_mu, tmp_sig2] = gp(opts.hyp,inffunc,meanfunc, ...
        covfunc,likfunc,[g_models{j+1}.X],[g_models{j+1}.Y],xvec(iik,:));
    h2 = 0.5*log(tmp_sig2);
    rk(iik) = h2 - h1; % importance is the difference between mutual information
end

processing distance: 100/9000
processing distance: 200/9000
processing distance: 300/9000
processing distance: 400/9000
processing distance: 500/9000
processing distance: 600/9000
processing distance: 700/9000
processing distance: 800/9000
processing distance: 900/9000
processing distance: 1000/9000
processing distance: 1100/9000
processing distance: 1200/9000
processing distance: 1300/9000
processing distance: 1400/9000
processing distance: 1500/9000
processing distance: 1600/9000
processing distance: 1700/9000
processing distance: 1800/9000
processing distance: 1900/9000
processing distance: 2000/9000
processing distance: 2100/9000
processing distance: 2200/9000
processing distance: 2300/9000
processing distance: 2400/9000
processing distance: 2500/9000
processing distance: 2600/9000
processing distance: 2700/9000
processing distance: 2800/9000
processing distance: 2900/9000
processing distance: 3000/9000
processing distance: 3100/9000
processing distance: 3200/9000
processing distan

In [22]:
for kj=1:length(grls) % test/validate different remaining percentage
    kj
    gr=grls(kj)

    % remove data, and use sparse approximation
    crk = rk;
    [~, crk_idx] = sort(crk, 'descend');
    rn = round(n*gr);
    global_index = crk_idx(1:rn);
    global_index = zeros(n,1);
    global_index(crk_idx(1:rn)) = 1; % select remaining data according to the importance
    
    vfe_opts.global_index = global_index;
    models = aggregation_train_GRBCM_VS_apx(xvec,yvec,idx,vfe_opts); % use hyp of vfe
    [tmu,ts2] = aggregation_predict_GRBCM_VS_apx(xvec_val,models,vfe_opts);
    if ynorm==1
        tmu = tmu * norm_fstd + norm_fmean;
        ts2 = ts2 * norm_fstd^2;
    end
    [MSE,SMSE,MSLL] = evaluate2(ori_xvec, ori_yvec, ori_xvec_val, ori_yvec_val, tmu, ts2);
    fprintf('%s (Dc size %d): \r\nMSE %6.8f, SMSE %6.8f, MSLL %6.8f\r\n', 'GRBCM++', n_per, MSE,SMSE,MSLL);
    vfe_gr_smse(ki,kj) = SMSE; vfe_gr_msll(ki,kj) = MSLL;
    if MSLL < best_msll
        best_msll = MSLL; best_msll_gr = gr;
    end
    if SMSE < best_smse
        best_smse = SMSE; best_smse_gr = gr;
    end
    
    sp_opts.global_index = global_index;
    models = aggregation_train_GRBCM_VS_apx(xvec,yvec,idx,sp_opts); % use hyp of vfe
    [tmu,ts2] = aggregation_predict_GRBCM_VS_apx(xvec_val,models,sp_opts);
    if ynorm==1
        tmu = tmu * norm_fstd + norm_fmean;
        ts2 = ts2 * norm_fstd^2;
    end
    [MSE,SMSE,MSLL] = evaluate2(ori_xvec, ori_yvec, ori_xvec_val, ori_yvec_val, tmu, ts2);
    fprintf('%s (Dc size %d): \r\nMSE %6.8f, SMSE %6.8f, MSLL %6.8f\r\n', 'GRBCM++', n_per, MSE,SMSE,MSLL);
    spgp_gr_smse(ki,kj) = SMSE; spgp_gr_msll(ki,kj) = MSLL;
end   

kj =  1
gr =  0.60000
GRBCM++ (Dc size 423): 
MSE 0.02861450, SMSE 0.02717819, MSLL -1.83767353
GRBCM++ (Dc size 423): 
MSE 0.03185021, SMSE 0.03025149, MSLL -1.80829879
kj =  2
gr =  0.61000
GRBCM++ (Dc size 423): 
MSE 0.02855101, SMSE 0.02711789, MSLL -1.84299221
GRBCM++ (Dc size 423): 
MSE 0.03165360, SMSE 0.03006474, MSLL -1.81388740
kj =  3
gr =  0.62000
GRBCM++ (Dc size 423): 
MSE 0.02828588, SMSE 0.02686607, MSLL -1.85232106
GRBCM++ (Dc size 423): 
MSE 0.03150171, SMSE 0.02992048, MSLL -1.81982214
kj =  4
gr =  0.63000
GRBCM++ (Dc size 423): 
MSE 0.02834215, SMSE 0.02691951, MSLL -1.85506885
GRBCM++ (Dc size 423): 
MSE 0.03138000, SMSE 0.02980487, MSLL -1.82395124
kj =  5
gr =  0.64000
GRBCM++ (Dc size 423): 
MSE 0.02829025, SMSE 0.02687021, MSLL -1.85938507
GRBCM++ (Dc size 423): 
MSE 0.03126966, SMSE 0.02970007, MSLL -1.82947121
kj =  6
gr =  0.65000
GRBCM++ (Dc size 423): 
MSE 0.02808970, SMSE 0.02667973, MSLL -1.86567039
GRBCM++ (Dc size 423): 
MSE 0.03104720, SMSE 0.0294887

In [23]:
fprintf('best msll gr %6.8f, best MSLL %6.8f\r\n', best_msll_gr, best_msll);
fprintf('best smse gr %6.8f, best SMSE %6.8f\r\n', best_smse_gr, best_smse);

best msll gr 0.99000000, best MSLL -1.98482877
best smse gr 0.97000000, best SMSE 0.02366612


In [24]:
for ki=1:kti
    for gr = [best_msll_gr, best_smse_gr, 1.0]
        gr
        crk = rk;
        crk(I_com) = -1e10;
        [~, crk_idx] = sort(crk, 'descend');
        rn = round(n*gr);
        global_index = zeros(n,1);
        global_index(crk_idx(1:rn)) = 1; % select remaining data according to the importance
           
        vfe_opts.global_index = global_index;
        models = aggregation_train_GRBCM_VS_apx(xvec,yvec,idx,vfe_opts); % use hyp of vfe
        [tmu,ts2] =  aggregation_predict_GRBCM_VS_apx(xvec_test,models,vfe_opts);
        if ynorm==1
            tmu = tmu * norm_fstd + norm_fmean;
            ts2 = ts2 * norm_fstd^2;
        end
        [MSE,SMSE,MSLL] = evaluate2(ori_xvec, ori_yvec, ori_xvec_test, ori_yvec_test, tmu, ts2);
        fprintf('%s (Dc size %d): \r\nMSE %6.8f, SMSE %6.4f, MSLL %6.4f\r\n', 'GRBCM++ (VFE)', n_per, MSE,SMSE,MSLL);
%         grbcm2_gr_smse(ki,kj) = SMSE; grbcm2_gr_msll(ki,kj) = MSLL;
        
        sp_opts.global_index = global_index;
        models = aggregation_train_GRBCM_VS_apx(xvec,yvec,idx,sp_opts); % use hyp of vfe
        [tmu,ts2] =  aggregation_predict_GRBCM_VS_apx(xvec_test,models,sp_opts);
        if ynorm==1
            tmu = tmu * norm_fstd + norm_fmean;
            ts2 = ts2 * norm_fstd^2;
        end
        [MSE,SMSE,MSLL] = evaluate2(ori_xvec, ori_yvec, ori_xvec_test, ori_yvec_test, tmu, ts2);
        fprintf('%s (Dc size %d): \r\nMSE %6.8f, SMSE %6.4f, MSLL %6.4f\r\n', 'GRBCM++ (SPGP)', n_per, MSE,SMSE,MSLL);
%         grbcm2_spgp_gr_smse(ki,kj) = SMSE; grbcm2_spgp_gr_msll(ki,kj) = MSLL;
    end   
end

gr =  0.99000
GRBCM++ (VFE) (Dc size 423): 
MSE 0.02234145, SMSE 0.0225, MSLL -2.0045
GRBCM++ (SPGP) (Dc size 423): 
MSE 0.02344650, SMSE 0.0236, MSLL -1.9751
gr =  0.97000
GRBCM++ (VFE) (Dc size 423): 
MSE 0.02252808, SMSE 0.0227, MSLL -2.0000
GRBCM++ (SPGP) (Dc size 423): 
MSE 0.02369750, SMSE 0.0239, MSLL -1.9696
gr =  1
GRBCM++ (VFE) (Dc size 423): 
MSE 0.02223942, SMSE 0.0224, MSLL -2.0075
GRBCM++ (SPGP) (Dc size 423): 
MSE 0.02328184, SMSE 0.0235, MSLL -1.9789


In [26]:
save(exp_name)